## Atomistic Molecular Dynamics: Bulk Liquids - SPC/E Water

Today's workshop session will involve performing atomistic molecular dynamics of bulk liquid systems. The first system we will consider is a bulk water system where we will use the extended simple point charge model (SPC/E).

### Water models

#### SPC/E

<img src="utils/spce-diagram.png" alt="SPCE Diagram" width="300px"/>
<img src="utils/spce-table.png" alt="SPCE Diagram" width="200px"/>
(images from http://www.sklogwiki.org/SklogWiki/index.php/SPC/E_model_of_water)

### Build the system

As in our prior workshops, we will use mBuild to construct our system. Here, we will import mBuild and define a class for a single water molecule, which we load from a PDB structure file.

In [ ]:
import mbuild as mb

class H2O(mb.Compound):
    def __init__(self):
        super(H2O, self).__init__()
        mb.load('utils/h2o.pdb', compound=self)

Before putting together the bulk system, let's take a look at an instance of our water molecule class to make sure things look reasonable.

In [ ]:
water = H2O()
water.visualize()

Now that we've verified our `H2O` class is good, we'll define a class for a box of water. We'll set this up so that we can toggle the number of water molecules via a `n_molecules` argument.

The recipe for this class is relatively straightfoward. We create an instance of the `H2O` class and use the `fill_box` function to fill a box with copies of this molecule at a density of 1000 kg/m^3. We also rename each molecule copy to `Water` as this speeds up the atom-typing process (whereby only a single copy will need to be atom-typed and those types can then be mapped onto all subsequent copies).

In [ ]:
class WaterBox(mb.Compound):
    """An box of water."""
    def __init__(self, n_molecules):
        """Initialize an WaterBox Compound.

        Parameters
        ----------
        n_molecules : int
            Number of molecules to place in the box
        """
        super(WaterBox, self).__init__()
        
        # Create a prototype for water using the class above
        water = H2O()

        # Fill a box with water at 1000 kg/m^3
        box_of_waters = mb.fill_box(compound=water, n_compounds=n_molecules,
                                    density=1000)
        
        # Rename all molecules to `water`, this speeds up the atom-typing process
        for child in box_of_waters.children:
            child.name = 'Water'
        self.add(box_of_waters)

Let's create a water box with 500 molecules and visualize.

In [ ]:
water_box = WaterBox(500)
water_box.visualize()

We are going to run the simulations today in GROMACS. Like HOOMD, GROMACS is an open-source molecular simulation code, and, like HOOMD, it is optimized for GPUs. GROMACS does not have the flexibility of HOOMD (only a few potential styles are supported for force field terms), but GROMACS has a dedicated team of full-time developers that have optimized the code to yield perhaps the best performance of any freely available molecular dynamics code.

GROMACS expects two data formats when setting up a simulation:
  - **GRO** format specifies particle coordinates and box information
  - **TOP** format specifies the topology and force field information

mBuild (via the ParmEd package) can write to both of these formats. First, we'll output our water box to GRO format. Because we are only writing positions and box information, we do not have to atom-type the system just yet. However, we will use the `residues` argument. Although originating from the protein community, we can think of residues as molecules. This does not really provide much advantage in writing the GRO file, but it will when we save our topology.

In [ ]:
water_box.save('spce.gro', overwrite=True, residues='Water')

Let's take a quick look at some parts of the GRO file we've just written.

In [ ]:
!head spce.gro

Following a header line, we see that the GRO format specifies the number of particles in the system, and then contains a line for each particle specifying the residue name, the atom name (in our case the element), the atom index (starting from 1...), and the XYZ coordinates (note that GROMACS uses nanometers as its distance unit).

In [ ]:
!tail spce.gro

At the end of the file we see three values: `2.46388    2.46388    2.46388`. These represent our three box lengths in nanometers.

Okay, now that we've checked out the GRO file, let's write our TOP file. This is where we need to define a force field that can be applied to our system. Since we're using the SPC/E water model, we will use a force field that corresponds to these model parameters. Let's check this out real quick:

In [ ]:
!cat spce.xml

Okay, now let's write our TOP file, again providing the `residues` argument, and now also providing a `forcefield_files` argument.

In [ ]:
water_box.save('spce.top', forcefield_files='spce.xml', overwrite=True, residues='Water')

### Run the simulation

Now we're ready to run a simulation! We'll split our simulation up into two stages: an equilibration stage and a production stage. Both of which will be performed under the NPT ensemble.

#### GROMACS MDP files

Unlike HOOMD, whose simulation script is in Python, GROMACS has its own format for defining simulation parameters. Specifically, these are housed within another file, called an MDP file (Molecular Dynamics Parameters). There are numerous commands available for you to fine-tune how you would like to run your simulation (see the complete list [here](http://manual.gromacs.org/online/mdp_opt.html)). Let's take a look at the MDP file we're going to be using for equilibration.

In [ ]:
!cat npt-eq.mdp

We'll walk through section-by-section to see what's going on. You should find many similarities between the sections in the GROMACS MDP file and the HOOMD run script. As you become more familiar with various simulation codes, you'll find that they all share many of the same qualities, each with its own unique spin on how simulations should be defined.

#### GROMPP-ing

In [ ]:
!gmx grompp -v -f npt-eq.mdp -c spce.gro -p spce.top -o npt-eq-spce

#### Equilibration

In [ ]:
!gmx mdrun -s npt-eq-spce.tpr -o -x -deffnm npt-eq-spce

Let's view the trajectory!

**NOTE:** If the following code block displays a blank screen, we will likely need to view our trajectory with VMD (outside of the notebook).

In [ ]:
import nglview as nv
import mdtraj as md

t = md.load("npt-eq-spce.xtc", top="npt-eq-spce.gro")
nv.show_mdtraj(t)

GROMACS contains many built-in utilities for analysis. Here, we will use the `gmx energy` utility to extract output from our simulation to a file in column format that we can read with Numpy and plot.

In [ ]:
!echo 6 7 8 12 13\\n0 | gmx energy -f npt-eq-spce.edr -o npt-eq-spce.xvg

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('npt-eq-spce.xvg', skiprows=27)

# Strip out the very beginning of the run
data = data[10:]

fig, ax = plt.subplots(5, 1)

properties = ['Total energy, kJ/mol', 'Temperature, K', 'Pressure, atm', 'Volume, nm^3', 'Density, kg/m^3']

for i, sub_ax in enumerate(ax):
    sub_ax.plot(data[:, 0], data[:, i + 1])
    sub_ax.set_title(properties[i])
    sub_ax.set_ylabel(properties[i])
    
fig.subplots_adjust(hspace=1.0)
fig.set_size_inches(4, 12)
plt.show()

#### NPT

In [ ]:
!gmx grompp -v -f npt-prod.mdp -c npt-eq-spce.gro -p spce.top -o npt-prod-spce

In [ ]:
!gmx mdrun -s npt-prod-spce.tpr -o -x -deffnm npt-prod-spce

View trajectory

In [ ]:
import nglview as nv
import mdtraj as md

t = md.load("npt-prod-spce.xtc", top="npt-prod-spce.gro")
nv.show_mdtraj(t)

In [ ]:
!echo 6 7 8 12 13\\n0 | gmx energy -f npt-prod-spce.edr -o npt-prod-spce.xvg

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('npt-prod-spce.xvg', skiprows=27)

fig, ax = plt.subplots(5, 1)

properties = ['Total energy, kJ/mol', 'Temperature, K', 'Pressure, atm', 'Volume, nm^3', 'Density, kg/m^3']

for i, sub_ax in enumerate(ax):
    sub_ax.plot(data[:, 0], data[:, i + 1])
    sub_ax.set_title(properties[i])
    sub_ax.set_ylabel(properties[i])
    
fig.subplots_adjust(hspace=1.0)
fig.set_size_inches(4, 12)
plt.show()